<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Deployment scaling with the Watson Machine Learning Python Client</b></font></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
   </tr>
</table>

This notebook demonstrates how to scale deployments using the Watson Machine Learning Python Client. 
A PMML model will be used in this example – you will first create and deploy the model, and then scale the deployment. 

Some familiarity with Python is helpful. This notebook uses `watson-machine-learning-client-V4` and is compatible with CP4D 3.0 and Python 3.6.


## Table of Contents

1.	[Set up the environment](#setup)<br>
2.	[Save the PMML model](#save)<br>
3.  [Create an online deployment](#deploy)<br>
4.  [Deployment scaling](#scale)<br>
5.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

To get started on CP4D, find documentation on installation and set up <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener no referrer">here</a>.

Import the `watson-machine-learning-client` module.
<div class="alert alert-block alert-info">
For more information about the <b>Watson Machine Learning Python client (V4)</b>, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">Python client documentation</a>. If you're using the notebook within a project on your CP4D cluster, you do not need to install this package as it comes pre-installed with the notebooks. The installation code below is for demonstration but is non-executable at this stage.
</div>

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Authenticate the Python client on CP4D.**
<div class="alert alert-block alert-info">To find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-authentication-local.html" target="_blank" rel="noopener no referrer">Documentation.</a></div>

In [2]:
# Enter your credentials here.

import sys,os,os.path
token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
     "instance_id": "openshift",
     #"token": token,
    "username": "---",
    "password": "---",
     "url": "---",
     "version": "3.0.0"
}

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
client.version

'1.0.64'

<div class="alert alert-block alert-info">
To save the model in the deployment space, you have to obtain the space UID of the deployment space you've created. Then you'd use this to set the default space using the python client. From there you'll be able to deploy and score the model in your deployment space.</div>

### To set the default space, follow these steps.

<div class="alert alert-block alert-info">
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Analytics deployments</b> under <b>Analyze</b> from the Navigation Menu on the top left of this page.</div>

Alternatively, you can create a deployment and obtain its UID using the code in the following cell. The cell is not executable cell at this stage, but you can enter the name of your space in the metadata and use it if needed.

In [5]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [6]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = 594b22d1-588c-4de0-9cbd-d98e6fa890e8


You can set the default space using the cell below.

In [7]:
client.set.default_space(space_uid)

'SUCCESS'

<a id="save"></a>
## 2. Save the PMML model

Download the sample PMML model, `iris_chaid.xml` from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

In [8]:
!wget https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml --output-document=iris_chaid.xml

--2020-05-12 05:14:01--  https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pmservice/wml-sample-models/master/pmml/iris-species/model/iris_chaid.xml [following]
--2020-05-12 05:14:01--  https://raw.githubusercontent.com/pmservice/wml-sample-models/master/pmml/iris-species/model/iris_chaid.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.68.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.68.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13876 (14K) [text/plain]
Saving to: ‘iris_chaid.xml’

100%[======================================>] 13,876      --.-K/s   in 0.07s   

2020-05-12 05:14:02 (204 KB/s) - ‘iris_chaid.xml’ saved [13876/13876]



Store the downloaded file as CHAID PMML model for Iris data, and then list all the saved models. First, you need to create the model metadata. The software specification required to save and deploy a PMML model is `spark-mllib_2.3`.

In [9]:
# Model metadata
software_spec_uid = client.software_specifications.get_uid_by_name('spark-mllib_2.3')
props_pmml = {
    client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: 'pmml_4.3'
}

<div class="alert alert-block alert-info">To list the supported software specifications, run <tt>client.software_specifications.list()</tt>.<br>To find more information about the frameworks with their respective <b>Types</b> and <b>Software Specifications</b>, visit the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/wmls/wmls-deploy-python-types.html" target="_blank" rel="noopener no referrer">documentation</a>.</div>

You can extract the model UID from the saved model details.

In [10]:
# Create the model artifact.
model_artifact = client.repository.store_model("iris_chaid.xml", meta_props=props_pmml)
model_uid = client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = f71d596f-1da5-4c9b-a051-3b97ce6321dd


You can list all stored models once again using the `list_models` method.

In [11]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  -----------------
GUID                                  NAME                             CREATED                   TYPE
f71d596f-1da5-4c9b-a051-3b97ce6321dd  CHAID PMML model for Iris data   2020-05-12T05:14:04.002Z  pmml_4.3
4a15c2ec-56a9-440b-a81c-5468e6b5f81b  SPSS model for Churn prediction  2020-05-12T04:04:21.002Z  spss-modeler_18.1
90a7aef0-2d46-49ed-994d-c065512828f2  SPSS model for Churn prediction  2020-05-12T03:56:45.002Z  spss-modeler_18.1
81a55ede-932a-44d5-86be-ef48cbed778d  Tensorflow custom library        2020-05-12T03:33:45.002Z  tensorflow_1.15
------------------------------------  -------------------------------  ------------------------  -----------------


<div class="alert alert-block alert-info">
From the list of stored models, you can see that model is successfully saved in the deployment space. You can view your model by selecting <b>Analytics Deployments</b> under <b>Analyze</b> from the Navigation Menu and clicking on your deployment space name.</div>

## 3. Create an online deployment<a id="deploy"></a>

Now, create an online deployment, *Iris species prediction*, for the stored model, and list all the online deployments for the model.

In [12]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Iris species prediction",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [13]:
# Create an online deployment.
deployment_details = client.deployments.create(model_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'f71d596f-1da5-4c9b-a051-3b97ce6321dd' started

#######################################################################################


initializing......................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='42cc7fe7-f872-474f-9002-bc8ad0481ee6'
------------------------------------------------------------------------------------------------




In [14]:
# List the deployments.
client.deployments.list()

------------------------------------  ------------------------------------  -----  ------------------------  -------------
GUID                                  NAME                                  STATE  CREATED                   ARTIFACT_TYPE
42cc7fe7-f872-474f-9002-bc8ad0481ee6  Iris species prediction               ready  2020-05-12T05:14:08.942Z  model
195c2bf6-6421-455e-afc0-592a591dafbf  My_Shiny_App_deployment               ready  2020-05-12T04:21:11.788Z  unknown
02d47cec-562c-4f63-8d87-f47c55246775  Sample SPSS model deployment          ready  2020-05-12T04:04:27.203Z  model
4de63cb0-974e-4a1f-8e6d-2ef18119528a  Sample SPSS model deployment          ready  2020-05-12T03:56:56.003Z  model
ec0c61f1-9134-48f4-a4e5-0feaa0efc60e  Tensorflow custom library deployment  ready  2020-05-12T03:33:51.111Z  model
------------------------------------  ------------------------------------  -----  ------------------------  -------------


<div class="alert alert-block alert-info">
From the list of deployed models, you can see that model was successfully deployed in the deployment space.</div>

Get the UID of the deployment.

In [15]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment UID = {}'.format(deployment_uid))

Deployment UID = 42cc7fe7-f872-474f-9002-bc8ad0481ee6


## 4. Deployment scaling <a id="scale"></a>

<a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-online.html#creating-multiple-copies-of-an-online-deployment" target="_blank" rel="noopener noreferrer">Deployment scaling</a> can only be performed on online deployments and <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-shiny-app-python.html#creating-multiple-copies-of-a-shiny-app-deployment" target="_blank" rel="noopener noreferrer">Shiny deployments</a>. When you deployed the model in the previous section, only one copy was created by default. In order to change the number of copies, you need to modify the deployment metadata, specifically the `HARDWARE_SPEC` field as shown below. The `num_nodes` field is where you specify the number of deployment copies. You must also populate either the `name` or `id` field under `HARDWARE_SPEC` as well. 

In this example, 3 deployment copies will be made. Both the `id` and `name` fields are specified, but as mentioned previously, only one of them is required.

In [16]:
hardware_spec_uid = client.hardware_specifications.get_uid_by_name("M")
change_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Iris species prediction update",
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name":"M",
        "id": hardware_spec_uid,
        "num_nodes": 3
    }
}

In [17]:
hardware_spec_uid

'c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24'

In [18]:
client.hardware_specifications.list()

-------------  ------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------
NAME           ID                                    DESCRIPTION
V100x2         a02f3ab5-6964-4f06-a870-c7cc69187895  A hardware specification providing 52 CPU cores and 96 GiB of memory with 2 Nvidia v100 GPUs.
L              a6c4923b-b8e4-444c-9f43-8a7ec3020110  A hardware specification providing 8 CPU cores and 32 GiB of memory.
Default Spark  ac59d20a-9c7c-4504-a853-788ef35969da  A hardware specification for Spark with 1 CPU and 4 GiB of memory for master and worker nodes, with 2 workers.
XXS            b128f957-581d-46d0-95b6-8af5cd5be580  A hardware specification providing one CPU core and 2 GiB of memory.
M-Spark        b2232f7a-bfad-4822-9bce-6ba1af49217a  A hardware specification for Spark service with 2 CPU and 8 GiB of memory for master and 4 CPU and 16 GiB of memory for worker nodes.
V100x

Use the following code to update the deployment using the deployment scaling metadata in the previous cell.

In [19]:
update_details = client.deployments.update(deployment_uid, change_meta)

You can see the status of the deployment scaling here.

In [20]:
client.deployments.get_details(deployment_uid)['entity']['status']['message']['text'].split(';')

['scaling_status: inprogress',
 'requested_copies: 3',
 'deployed_copies: 1',
 'more_info: ']

Once the `scaling_status` updates to `completed` when you run line of code above, the deployment has successfully been scaled.

In [21]:
client.deployments.list()

------------------------------------  ------------------------------------  -----  ------------------------  -------------
GUID                                  NAME                                  STATE  CREATED                   ARTIFACT_TYPE
42cc7fe7-f872-474f-9002-bc8ad0481ee6  Iris species prediction update        ready  2020-05-12T05:14:08.942Z  model
195c2bf6-6421-455e-afc0-592a591dafbf  My_Shiny_App_deployment               ready  2020-05-12T04:21:11.788Z  unknown
02d47cec-562c-4f63-8d87-f47c55246775  Sample SPSS model deployment          ready  2020-05-12T04:04:27.203Z  model
4de63cb0-974e-4a1f-8e6d-2ef18119528a  Sample SPSS model deployment          ready  2020-05-12T03:56:56.003Z  model
ec0c61f1-9134-48f4-a4e5-0feaa0efc60e  Tensorflow custom library deployment  ready  2020-05-12T03:33:51.111Z  model
------------------------------------  ------------------------------------  -----  ------------------------  -------------


<a id="summary"></a>
## 5. Summary and next steps

You successfully completed this notebook! You learned how to use Watson Machine Learning to scale a deployment.

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success">

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">CP4D 3.0</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 <li> <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener noreferrer">Sample models</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a><br></li>
</div>

### Citation

Dua, D. and Karra Taniskidou, E. (2017). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.

### Authors

**Wojciech Sobala** is a Data Scientist at IBM.  <br><br>
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2018-2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>